In [2]:
import pandas as pd
import pyodbc
import glob


In [8]:

# DB connection
conn = pyodbc.connect(
    'DRIVER={SQL Server};SERVER=DESKTOP-32I8LFV;DATABASE=CensusData;Trusted_Connection=yes;'
)
cursor = conn.cursor()


In [17]:
import pandas as pd

def safe_val(val):
    return None if pd.isna(val) else val

def insert_data(row):
    year = int(safe_val(row['year']))
    StatFP = str(safe_val(row['state'])).zfill(2)
    CountyFP = str(safe_val(row['county'])).zfill(3)
    Tract = str(safe_val(row['tract'])).zfill(6)

    # === GEOID_INFO ===
    cursor.execute("""
        IF NOT EXISTS (
            SELECT 1 FROM Geoid_info WHERE Year=? AND StatFP=? AND CountyFP=? AND Tract=?
        )
        INSERT INTO Geoid_info (Year, StatFP, CountyFP, Tract)
        VALUES (?, ?, ?, ?)
    """, year, StatFP, CountyFP, Tract, year, StatFP, CountyFP, Tract)

    # === POPULATION_DEMOGRAPHICS ===
    cursor.execute("""
        IF NOT EXISTS (
            SELECT 1 FROM Population_Demographics WHERE Year=? AND StatFP=? AND CountyFP=? AND Tract=?
        )
        INSERT INTO Population_Demographics
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, year, StatFP, CountyFP, Tract,
         year, StatFP, CountyFP, Tract,
         safe_val(row['Median Age']),
         safe_val(row['White']),
         safe_val(row['Black or African American']),
         safe_val(row['Hispanic or Latino']),
         safe_val(row['Asian']),
         safe_val(row['American Indian/Alaska Native']),
         safe_val(row['Native Hawaiian & Pacific Islander']),
         safe_val(row['Total Male Population']),
         safe_val(row['Total Female Population']),
         safe_val(row['With Disability']),
         safe_val(row['With Health Insurance']))

    # === SOCIOECONOMIC_FACTORS ===
    cursor.execute("""
        IF NOT EXISTS (
            SELECT 1 FROM Socioeconomic_Factors WHERE Year=? AND StatFP=? AND CountyFP=? AND Tract=?
        )
        INSERT INTO Socioeconomic_Factors
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, year, StatFP, CountyFP, Tract,
         year, StatFP, CountyFP, Tract,
         safe_val(row['Median Household Income']),
         safe_val(row['Per Capita Income']),
         safe_val(row['People Below Poverty Line']),
         safe_val(row['Child Poverty Rate']),
         safe_val(row['High School Graduate']),
         safe_val(row['Some College']),
         safe_val(row["Bachelor’s Degree or Higher"]),
         safe_val(row['In Labor Force']),
         safe_val(row['Civilian Labor Force']),
         safe_val(row['Employed']),
         safe_val(row['Unemployed']),
         safe_val(row['Not in Labor Force']))

    # === HOUSING_DATA ===
    cursor.execute("""
        IF NOT EXISTS (
            SELECT 1 FROM Housing_Data WHERE Year=? AND StatFP=? AND CountyFP=? AND Tract=?
        )
        INSERT INTO Housing_Data
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, year, StatFP, CountyFP, Tract,
         year, StatFP, CountyFP, Tract,
         safe_val(row['Vacant Housing Units']),
         safe_val(row['Owner-Occupied Units']),
         safe_val(row['Renter-Occupied Units']),
         safe_val(row['Median Home Value']),
         safe_val(row['Median Gross Rent']),
         safe_val(row['Household Size']))

    # === TRANSPORTATION_MOBILITY ===
    cursor.execute("""
        IF NOT EXISTS (
            SELECT 1 FROM Transportation_Mobility WHERE Year=? AND StatFP=? AND CountyFP=? AND Tract=?
        )
        INSERT INTO Transportation_Mobility
        VALUES (?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)
    """, year, StatFP, CountyFP, Tract,
         year, StatFP, CountyFP, Tract,
         safe_val(row['Worked at Home']),
         safe_val(row['Public Transportation']),
         safe_val(row['Average Travel Time to Work']),
         safe_val(row['No Vehicle Available']),
         safe_val(row['1 Vehicle Available']),
         safe_val(row['2 Vehicles Available']),
         safe_val(row['3 or More Vehicles Available']),
         safe_val(row['Car Ownership per Household']))

    conn.commit()


In [18]:

# Loop over each CSV
for file in glob.glob("acs_race_population_*.csv"):
    print(f"Inserting: {file}")
    df = pd.read_csv(file)

    # Fix columns for clean referencing
    df.columns = df.columns.str.strip()

    for _, row in df.iterrows():
        insert_data(row)

print("✅ All data inserted.")

Inserting: acs_race_population_2012.csv
Inserting: acs_race_population_2013.csv
Inserting: acs_race_population_2014.csv
Inserting: acs_race_population_2015.csv


KeyboardInterrupt: 

In [ ]:
Bachelor’s Degree or Higher